In [4]:
import hoomd
from hoomd import md
import numpy as np
import matplotlib.pyplot as plt
import hoomd.htf as htf
import tensorflow as tf

In [90]:
class WCAPotential(htf.SimModel):
    def setup(self):
        self.avg_rdf = tf.keras.metrics.MeanTensor()
    def compute(self, nlist):
        # Use Weeks-Chandler-Anderson (WCA) repulisve potential
        r12 = htf.nlist_rinv(nlist)**12 # nlist_rinv is neighbor 1 / r^12
        # make it so anything above 2^1/6 is 0
        r = tf.norm(nlist[:,:,:3], axis=2)
        pair_energy = tf.cast(r < 2**(1/6), tf.float32) * r12
        particle_energy = tf.reduce_sum(pair_energy, axis=1) # sum over neighbors        
        forces = htf.compute_nlist_forces(nlist, particle_energy)
        # compute rdf
        inst_rdf = htf.compute_rdf(nlist, [0, 3.5])
        self.avg_rdf.update_state(inst_rdf)
        return forces


In [111]:
########### Hoomd-Sim Code ################
hoomd.context.initialize('--mode=cpu')


phi = 0.3 # packing fraction
sigma = 1
N = 125
L = (N*(np.pi*0.125*4/3*sigma**3)/phi)**(1./3.)
print(L)


6.019990328451137


In [112]:
#%%
box = hoomd.data.boxdim(L=L,dimensions=3)
snap = hoomd.data.make_snapshot(N=N,particle_types=['A'],box=box)
a = L/(N**(1/3))

snap.particles.position[:] = [(i,j,k) for i in np.arange(-L/2,L/2,a) for j in np.arange(-L/2,L/2,a) for k in np.arange(-L/2,L/2,a)]
print(a)
print(snap.particles.position)

1.2039980656902276
[[-3.0099952  -3.0099952  -3.0099952 ]
 [-3.0099952  -3.0099952  -1.8059971 ]
 [-3.0099952  -3.0099952  -0.60199904]
 [-3.0099952  -3.0099952   0.60199904]
 [-3.0099952  -3.0099952   1.8059971 ]
 [-3.0099952  -1.8059971  -3.0099952 ]
 [-3.0099952  -1.8059971  -1.8059971 ]
 [-3.0099952  -1.8059971  -0.60199904]
 [-3.0099952  -1.8059971   0.60199904]
 [-3.0099952  -1.8059971   1.8059971 ]
 [-3.0099952  -0.60199904 -3.0099952 ]
 [-3.0099952  -0.60199904 -1.8059971 ]
 [-3.0099952  -0.60199904 -0.60199904]
 [-3.0099952  -0.60199904  0.60199904]
 [-3.0099952  -0.60199904  1.8059971 ]
 [-3.0099952   0.60199904 -3.0099952 ]
 [-3.0099952   0.60199904 -1.8059971 ]
 [-3.0099952   0.60199904 -0.60199904]
 [-3.0099952   0.60199904  0.60199904]
 [-3.0099952   0.60199904  1.8059971 ]
 [-3.0099952   1.8059971  -3.0099952 ]
 [-3.0099952   1.8059971  -1.8059971 ]
 [-3.0099952   1.8059971  -0.60199904]
 [-3.0099952   1.8059971   0.60199904]
 [-3.0099952   1.8059971   1.8059971 ]
 [-1.8

In [113]:

# create a square lattice
system = hoomd.init.read_snapshot(snap)
nlist = hoomd.md.nlist.cell()
model = WCAPotential(64)
tfcompute = htf.tfcompute(model)

gsd = hoomd.dump.gsd('125_snap.gsd',period=100,group=hoomd.group.all(),overwrite=True)
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nvt(group=hoomd.group.all(), kT=0.5, tau=0.5).randomize_velocities(seed=1)

tfcompute.attach(nlist, r_cut=1.5)

# NVT ensemble with starting temperature of 1

#equilibrate
hoomd.run(10000)




notice(2): Group "all" created containing 125 particles
notice(2): Force mode is FORCE_MODE.tf2hoomd 
notice(2): Starting TensorflowCompute 
notice(2): completed reallocate
notice(2): Setting flag indicating virial modification will occur
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 5937 / 10000 | TPS 593.627 | ETA 00:00:06
Time 00:00:16 | Step 10000 / 10000 | TPS 625.277 | ETA 00:00:00
Average TPS: 606.07
---------
-- Neighborlist stats:
547 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 10 / n_neigh_max: 19 / n_neigh_avg: 15.136
shortest rebuild period: 10
-- Cell list stats:
Dimension: 3, 3, 3
n_min    : 3 / n_max: 7 / n_avg: 4.62963
** run complete **


In [115]:
dir(nlist)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_exclusion',
 'cpp_cl',
 'cpp_nlist',
 'cur_id',
 'exclusion_list',
 'exclusions',
 'is_exclusion_overridden',
 'name',
 'query_update_period',
 'r_buff',
 'r_cut',
 'reset_exclusions',
 'set_params',
 'subscribe',
 'subscriber_callbacks',
 'tune',
 'update_exclusions_defaults',
 'update_rcut']

In [ ]:






# reset rdf statistics
model.avg_rdf.reset_states()

hoomd.run(50000)